In [ ]:
# !pip install python-dotenv OpenAI fastapi uvicorn nest_asyncio pydantic tiktoken --quiet

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI(
    api_key=os.environ.get("GILAS_API_KEY"),
    base_url="https://api.gilas.io/v1/"
)

In [ ]:
models = ["ft:gpt-4o-mini:xxxx:zzzzz",
          "gpt-4o-mini"]

In [ ]:
max_tokens = 4095

def get_completion(instruction, prompt, model):
    messages = [{"role": "system", "content": instruction},
                {"role": "user", "content": prompt}]
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

In [ ]:
instruction = f"""
You are an intelligent translator specializing in translating technical texts from English to Farsi. Please follow these steps to translate the provided English texts to Farsi:
1. Thoroughly understand the given text.
2. Translate the content into fluent Farsi, preserving the original structure and flow.
3. Tailor the translation for software engineers by keeping the technical and programming terms in English, enclosed in backticks (e.g., `GPT-3`).
4. You can include additional explanations in Farsi for clarity, if needed.
5. If you encounter a block of text enclosed in triple backticks, do not translate it; keep it as it is in your translation.
6. Review your translation to ensure you followed the steps. If the translated text is not fluent Farsi, revise it. 
7. Your output should only include the very final version and formatted translation. 
"""

In [ ]:
def get_next_test_data():
    file_path = "test-data.txt"
    with open(file_path, "r") as file:
        lines = file.readlines()
    
    if not hasattr(get_next_test_data, "current_index"):
        get_next_test_data.current_index = 0

    while get_next_test_data.current_index < len(lines):
        next_line = lines[get_next_test_data.current_index].strip()
        get_next_test_data.current_index += 1
        if next_line:
            return next_line

    return None

In [ ]:
def generate_html_table(prompt, model_responses):
    # Initialize the table with headers if it's the first call
    if not hasattr(generate_html_table, 'header_added'):
        generate_html_table.html_content = f"<table>\n<tr><th>English Text</th>"
        for model_name in model_responses:
            generate_html_table.html_content += f"<th>{model_name}</th>"
        generate_html_table.html_content += "</tr>\n"
        generate_html_table.header_added = True
    else:
        # Remove the closing </table> tag if it exists
        if generate_html_table.html_content.endswith("</table>"):
            generate_html_table.html_content = generate_html_table.html_content[:-8]


    # Append new row to the html content
    generate_html_table.html_content += f"<tr><td>{prompt}</td>"
    for response in model_responses.values():
        generate_html_table.html_content += f"""<td><input type="checkbox"><label dir="rtl">{response}</label></td>"""
    generate_html_table.html_content += "</tr>\n"

    # Re-add the closing </table> tag
    generate_html_table.html_content += "</table>"

    return generate_html_table.html_content


In [ ]:
import asyncio

async def gen_prompt_response(sample):
    # Create a list of tasks to run synchronous functions in separate threads
    tasks = [asyncio.to_thread(get_completion, instruction, sample, model_name) for model_name in models]
    
    # Wait for all tasks to complete
    responses = await asyncio.gather(*tasks)

    # Pair model names with their respective responses
    model_responses = {model_name: response for model_name, response in zip(models, responses)}
    
    # Generate the HTML table with the results
    return generate_html_table(sample, model_responses)

In [ ]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
import uvicorn
import nest_asyncio
from typing import List, Dict

app = FastAPI()

@app.get("/", response_class=HTMLResponse)
async def prompt_endpoint():
    html_table = ""
    sample = get_next_test_data()
    while sample:
        html_table = await gen_prompt_response(sample)
        sample = get_next_test_data()

    
    html_content = f"""<html>
    <head><title>Prompt Results</title>
    </head>
    
    <body> 
    {html_table} 
    </body></html>"""


    return html_content

  
# If using asyncio in Jupyter, apply the following:
nest_asyncio.apply()

uvicorn.run(app, host="127.0.0.1", port=8000, log_level="info")